# CJ  - DATA EXTRACTING

브랜드별 카드소비 점유율? 브랜드마다 카드점유율을 보여달라!

### [전체과정]
1. 데이터가져오기 (RETREIVE MONTHLY DATA)

 - (where) exclude : negative payment 
 - revisit_his : addr, card_name, cnt_pay
 - inner join : company_id(co_name, cate, cate1)
 * cardname, co_name, addr - refining required
 
2. 전처리

 - 정제<BR/>
     1) 브랜드의 카드점유율을 보려면 payment가 -0인 데이터를 제거해야 한다. <BR/>
        ( 중복된 데이터는 카드점유율을 과대추산한다.)<BR/>
     (ex) 0보다 작은 payment 제거 2832826 -> 2734284

 - 필요한 카테고리(브랜드)에 해당하는 데이터만 추출하기(MERGE & PICKout)<BR/>
    1) 카드사별 분류가 필요함. 정규식적용 <br/>
    2) 브랜드별 정규식 적용.<br/>
    3) 브랜드 x 카드사 의 matrix 생성<br/>
 
3. 브랜드별/카드사별 데이터 count하기(SUM+GROUP BY)
4. 그래프로 시각화하기



In [2]:
from sqlalchemy import create_engine
import pymysql, pandas as pd
pymysql.install_as_MySQLdb()
import MySQLdb,pyarrow

# 1. Retreive data

### REVISIT HIS DATA - 우선순위 1번째

In [20]:
p1= pd.read_pickle('data/revisit_his_p1.pick')
p2= pd.read_pickle('data/revisit_his_p2.pick')
p3= pd.read_pickle('data/revisit_his_p3.pick')
p4= pd.read_pickle('data/revisit_his_p4.pick')
p5= pd.read_pickle('data/revisit_his_p5.pick')

data = pd.concat([p1,p2,p3,p4,p5])
data['PAYMENT']=data['PAYMENT'].astype(int)
print(data.shape," would be reduced to ",data[data['PAYMENT']>0].shape)
print(data.drop_duplicates('COMPANY_ID').shape)
data.head()

(2832826, 4)  would be reduced to  (2734284, 4)
(458269, 4)


,EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT
0,eppcn15a40d5c3cb6ada7e6ac607ec,5940135,KB국민카드(3*4*),11980
1,eppcn159a64276cb6ada7762f7ee01,5791118,신한카드(5*8*),6500
2,eppcn15a250c93cb6ada272ce5e77e,14366,현대카드ZERO,12000
3,eppcn159f69615cb6ada0c9b259456,5434086,KB국민카드(1*1*),61200
4,eppcn15a1512d6cb6ada72bfb2273d,7112451,KB국민카드(9*5*),4500


### COMPANY INFO  - 우선순위 1번째

In [26]:
company=pd.read_pickle('data/company_true.pick')
print(company.shape)

(458268, 5)


# 2. Preprocessing 


## Refine
 - TAKE DATA with the condition to the column, "PAYMENT", which has nonnegative values  
    the purpose of this coding is to get what portion of market-share each brand takes !
    Hence, the canceled data should not be counted becaue the customer having used that store just tried "ONCE" to buy a product of.
 - Clean cardname data

In [22]:
# data[:100].to_parquet('data/test')
data =data[data['PAYMENT']>0]
data.shape
data.head(3)

,EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT
0,eppcn15a40d5c3cb6ada7e6ac607ec,5940135,KB국민카드(3*4*),11980
1,eppcn159a64276cb6ada7762f7ee01,5791118,신한카드(5*8*),6500
2,eppcn15a250c93cb6ada272ce5e77e,14366,현대카드ZERO,12000


In [23]:
def cleanCardName(sentence):
    import re
    pattern = re.compile(r'\(.{0,4}\)|re\)|\<현장즉시결제\>|\[|\s|카드|\<|\>|\'"')
    sentence = re.sub(pattern,'',sentence.lower())
    return sentence # return 안쓰면 none값 뜬다.
    
data = data.assign( CARD_NAME = data['CARD_NAME'].apply(cleanCardName) )


## Merge : revisit + company
 for getting Store information (name, cate, addr)

In [30]:
data_comp = pd.merge(data,company[company.BRAND!='na'],how='inner',on='COMPANY_ID')
print(data_comp.shape)
data_comp.head(3)

(142530, 8)


,EPOPCON_ID,COMPANY_ID,CARD_NAME,PAYMENT,CO_NAME,CATE,CATE1,BRAND
0,eppcn15a2ab595cb6ada272ce5ed16,6631842,농협bc,7000,휴,카페/디저트,카페,커피기타
1,eppcn15a0115e6cb6ada0c9b259b5e,306820,현대thered,6700,롯데시네마광명아울렛,문화,문화,롯데시네마
2,eppcn15a4a2ea2cb6ada16b6ab50df,306820,현대,8000,롯데시네마광명아울렛,문화,문화,롯데시네마


## Merge : datetime only related to necessary brands

In [52]:
brand= pd.read_excel('BRANDCODE.xlsx')
brand.columns=['GROUP','GRUOP_NAME','CODE','CODE_NAME','SYN','STOP']
data_comp_b = data_comp.merge(brand,how='inner',left_on='BRAND',right_on='CODE_NAME')[['EPOPCON_ID','CARD_NAME','BRAND','PAYMENT','GROUP','CODE']]
print(data_comp_b.shape)
data_comp_b.head(3)

(142530, 6)


,EPOPCON_ID,CARD_NAME,BRAND,PAYMENT,GROUP,CODE
0,eppcn15a2ab595cb6ada272ce5ed16,농협bc,커피기타,7000,2,999
1,eppcn15a08e902cb6ada0c9b259fc8,신한,커피기타,69800,2,999
2,eppcn15a46952a8c0c241b8a0194ae,삼성,커피기타,219000,2,999


## Naming CardName

In [53]:
test = data_comp_b#[['EPOPCON_ID','CARD_NAME']]
card = pd.read_excel('RULEofCARDNAME.xlsx')
card.columns=['corp','name_regx','prod_regx']
card = card.fillna('{null}')

for index, obj in card.iterrows():
    value = obj['name_regx']
    if obj['prod_regx'] =='{null}': 
        test.loc[test.CARD_NAME.str.contains(value),'CARD']=  obj['corp']
#         test[obj['코드명']]= test.CO_NAME.str.contains(value).astype(int)
    else:
        test.loc[~test.CARD_NAME.str.contains(obj['prod_regx'])&test.CARD_NAME.str.contains(value),'CARD']=  obj['corp']

print(test[test.CARD.isnull()].shape,test[~test.CARD.isnull()].shape)

C:\Users\epopc\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if sys.path[0] == '':


(23279, 7) (119251, 7)


In [54]:
data_comp_b = test

In [63]:
test[test.CARD=="{null}"]

,EPOPCON_ID,CARD_NAME,BRAND,PAYMENT,GROUP,CODE,CARD


In [65]:
test[test.CARD=="포인트"]

,EPOPCON_ID,CARD_NAME,BRAND,PAYMENT,GROUP,CODE,CARD
1495,eppcn15b052305cb6ada7988e15665,hi-point/475,커피기타,76600,2,999,포인트
2268,eppcn15a45c0678c0c241b8a01940b,hi-point/574,커피기타,8400,2,999,포인트
25448,eppcn15abf657acb6ada7dfc25387c,cjone/248,커피기타,3300,2,999,포인트
39198,eppcn15aabcc228c0c243d38b38014,hi-point/852,커피기타,9000,2,999,포인트
39199,eppcn15aabcc228c0c243d38b38014,hi-point/852,커피기타,9000,2,999,포인트
40394,eppcn15abf657acb6ada7dfc25387c,cjone/248,커피기타,3500,2,999,포인트
55671,eppcn15a6f127d8c0c243d38b36b34,kiaredmembersplatinumplus/450*,커피기타,34300,2,999,포인트
79371,eppcn15a45c0678c0c241b8a01940b,hi-point/574,롯데시네마,11000,1,1,포인트
93246,eppcn15aabcc228c0c243d38b38014,hi-point/852,할리스,4600,2,7,포인트
95581,eppcn15abf657acb6ada7dfc25387c,cjone/248,올리브영,11590,4,0,포인트


### SUM/GROUP_BY + RENAME_COLUMNS

 - 영화, 커피, 화장품 별로 데이터나누기

In [83]:
movie = data_comp_b[data_comp_b['GROUP']==1]
coffee = data_comp_b[data_comp_b['GROUP']==2]
beauty = data_comp_b[data_comp_b['GROUP']==4]
print(data_comp_b.shape, movie.shape, coffee.shape, beauty.shape)#(142530, 5) (12033, 5) (109292, 5) (21205, 5)
beauty.head()

(142530, 7) (12033, 7) (109292, 7) (21205, 7)


,EPOPCON_ID,CARD_NAME,BRAND,PAYMENT,GROUP,CODE,CARD
93512,eppcn159fb0996cb6ada0c9b2597cd,삼성,올리브영,22000,4,0,삼성
93513,eppcn159fb0996cb6ada0c9b2597cd,삼성,올리브영,22000,4,0,삼성
93514,eppcn159fb0996cb6ada0c9b2597cd,삼성,올리브영,22000,4,0,삼성
93515,eppcn1596bc35ecb6ada77a7319355,하나,올리브영,8000,4,0,하나
93516,eppcn159c0fafdcb6ada7762f7efea,삼성,올리브영,19560,4,0,삼성


In [87]:
import gc
gc.collect()
def make_stat_df(df):
    p_group = df[['BRAND','CARD','PAYMENT']].groupby(['BRAND','CARD']).agg(['count','sum']).reset_index()
    p_group.columns = ['BRAND','CARD','CNT_PAY','SUM_PAY'] 

#     print(p_group.shape) 
#     p_group.head()

    p_group1 = df[['BRAND','PAYMENT']].groupby(['BRAND']).agg(['count','sum']).reset_index()
    p_group1.columns = ['BRAND','BRAND_CNT_PAY','BRAND_SUM_PAY']
#     p_group1.head()

    p_stat = pd.merge(p_group,p_group1,how="inner",on='BRAND')
    p_stat = p_stat.assign(PCT_CNT=p_stat.apply(lambda x: round(x['CNT_PAY']/x['BRAND_CNT_PAY'],4)*100, axis=1))
    p_stat['PCT_SUM'] = p_stat.apply(lambda x: round(x['SUM_PAY']/x['BRAND_SUM_PAY'],4)*100, axis=1)
    return p_stat
# make_stat_df(movie).head(52)
# make_stat_df(coffee).head(52)
make_stat_df(beauty).head(52)

,BRAND,CARD,CNT_PAY,SUM_PAY,BRAND_CNT_PAY,BRAND_SUM_PAY,PCT_CNT,PCT_SUM
0,ARITAUM,BNK,26,715500,1105,25968590,2.35,2.76
1,ARITAUM,광주,5,37700,1105,25968590,0.45,0.15
2,ARITAUM,기업,30,708060,1105,25968590,2.71,2.73
3,ARITAUM,농협,183,4820040,1105,25968590,16.56,18.56
4,ARITAUM,대구,21,554600,1105,25968590,1.90,2.14
5,ARITAUM,롯데,67,1563740,1105,25968590,6.06,6.02
6,ARITAUM,삼성,96,2801530,1105,25968590,8.69,10.79
7,ARITAUM,신한,167,3559600,1105,25968590,15.11,13.71
8,ARITAUM,신협,5,58000,1105,25968590,0.45,0.22
9,ARITAUM,씨티,2,14300,1105,25968590,0.18,0.06


In [2]:
# p.to_pickle('april_data/hana_april.pick')

p = pd.read_pickle('april_data/hana_april.pick')

### EXPORT XML FILE 

- UnicodeDecode Error : illegal multibyte sequence

p=p.replace('\\xa0', '')  ->  with open(filename, mode, encoding='utf8') as f:

In [4]:
#https://stackoverflow.com/questions/18574108/how-do-convert-a-pandas-dataframe-to-xml

def to_xml(df, filename=None, mode='w'):
    bigxml=['<?xml version="1.0"?>','<resultset xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">']
    def row_to_xml(row):
        xml = [' <row>']
        for i, col_name in enumerate(row.index):
            xml.append('  <field name="{0}">{1}</field>'.format(col_name, row.iloc[i]))
        xml.append(' </row>')
        return '\n'.join(xml)
    resultset = '\n'.join(df.apply(row_to_xml, axis=1))
    bigxml.append(resultset)
    bigxml.append('</resultset>')
    res = '\n'.join(bigxml)
    
    if filename is None:
        return res
    with open(filename, mode, encoding='utf8') as f:
        f.write(res)

pd.DataFrame.to_xml = to_xml

#tmp.to_xml('test.csv')
# p.iloc[:2500000,].to_xml('april_data/hana_april1.xml')
# p.iloc[2500000:5000000,].to_xml('april_data/hana_april2.xml')
# p.iloc[5000000:7500000,].to_xml('april_data/hana_april3.xml')
p.iloc[7500000:,].to_xml('april_data/hana_april4.xml')

In [10]:
p.iloc[1006946:,].head()

,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
1006946,15a3991ec71e42756fbe86b6e,30188,KB국민,1,9200.0,경기도 수원시 장안구 정자동 535-12,마트,편의점,CU 청솔번영점,37.2985548,126.9872756
1006947,15a3991ec71e42756fbe86b6e,8077641,KB국민체크,1,7400.0,경기 수원시 장안구 천천동 527-1,마트,시장,장독대천천동점,37.2985548,126.9872756
1006948,15a3991ec71e42756fbe86b6e,7486252,KB국민,1,5100.0,경기도 수원시 장안구 정자동 877-4 정연메이져 1층,카페/디저트,베이커리,뚜레쥬르수원정자점,37.2985548,126.9872756
1006949,15a3991ec71e42756fbe86b6e,5256211,KB국민,1,2200.0,,뷰티,화장품,올리브영 수원천천점,37.2985548,126.9872756
1006950,15a3991ec71e42756fbe86b6e,5256211,KB국민체크,1,6210.0,,뷰티,화장품,올리브영 수원천천점,37.2985548,126.9872756


In [16]:
print(p[p['CO_NAME'].str.contains('\&|\<|\>|\'|\"')].shape)
print(p[p['CO_NAME'].str.contains('화장실')].shape)

(0, 11)

In [17]:
p[p['ADDR'].str.contains('&|<|>|\'|"')].shape
p[p['ADDR'].str.contains('@')].shape
p[p['ADDR'].str.contains('@')].head()

,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,CO_NAME,CATE,CATE1,ADDR,LATITUDE1,LONGITUDE1
7813,15a38c4da71e42756fbe6acf3,331209,롯데,1,1400.0,맥도날드 중앙대점,외식,패스트푸드,서울 동작구 흑석동 221 중앙대학교 약학대학 R@D센터 지하1층,37.583560,127.049909
8300,15a38c60de946d9170e4ac3ab,24010,KB국민,2,35000.0,CGV 인천점,문화,영화,인천 남동구 구월동 1130 SEE@SEEE빌딩 4층,37.540170,126.734375
8445,15a38c6516259ad0a9e318a81,1059333,신한BC,1,58000.0,호수세탁소,생활,,경기도 고양시 일산동구 장항동 875 호수4단지 LG@롯데아파트 상가,37.651921,126.775700
9280,15a38c7ec6259ad0a9e318c65,147219,KB국민,1,3600.0,김기택내과의원,의료/건강,병원,경기 김포시 장기동 494-31 C@C빌딩 5층~7층,37.505473,127.037884
12863,15a38cf52e946d9170e4acaa3,24010,KB국민,1,19000.0,CGV 인천점,문화,영화,인천 남동구 구월동 1130 SEE@SEEE빌딩 4층,37.468214,126.645818


In [7]:
print(p[p['ADDR'].str.contains('["\'&<>]')].shape)
print(p[p['CO_NAME'].str.contains('["\'&<>]')].shape)
print(p[p['CO_NAME'].str.contains('화장실')].shape)
p[p['CARD_NAME'].str.contains('["\'&<>]')].shape

(0, 11)
(0, 11)
(0, 11)


(0, 11)

In [6]:
p[p['ADDR'].str.contains('["\'#&<>]')].head()

,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,CO_NAME,CATE,CATE1,ADDR,LATITUDE1,LONGITUDE1
8886,15a38c74e6259ad0a9e318b8b,7691644,삼성,1,64968.0,AGODA,여행,숙박,"30 Cecil Street, Prudential Tower #19-08, Sing...",37.204112,127.061057
9253,15a38c7ec6259ad0a9e318c65,7691644,KB국민,1,83082.0,AGODA,여행,숙박,"30 Cecil Street, Prudential Tower #19-08, Sing...",37.505473,127.037884
14487,15a38d1a3e946d9170e4acc44,7691644,SC은행BC,3,880982.0,AGODA,여행,숙박,"30 Cecil Street, Prudential Tower #19-08, Sing...",37.566703,126.978313
15609,15a38d365e946d9170e4ace56,7691644,신한,2,341999.0,AGODA,여행,숙박,"30 Cecil Street, Prudential Tower #19-08, Sing...",37.500572,127.037400
23565,15a38dc40e946d9170e4adca6,7691644,하나,1,1140.0,AGODA,여행,숙박,"30 Cecil Street, Prudential Tower #19-08, Sing...",37.638295,126.877622


In [6]:
len(p.COMPANY_ID.unique())

846964